<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [178]:
import pandas as pd
import psycopg2
import plotly.express as px

In [180]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [181]:
# текст запроса
query_3_1 = f''' select COUNT(*)
                from vacancies
            '''

In [182]:
df_3_1 = pd.read_sql_query(query_3_1,connection)
df_3_1

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [183]:
query_3_2 = f'''select count(*)
from employers'''

In [184]:
df_3_2 = pd.read_sql_query(query_3_2,connection)
df_3_2

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [185]:
query_3_3 = f'''select count(*)
from areas'''

In [186]:
df_3_3 = pd.read_sql_query(query_3_3,connection)
df_3_3

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [187]:
query_3_4 = f'''select count(*)
from industries'''

In [188]:
df_3_4 = pd.read_sql_query(query_3_4,connection)
df_3_4

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,294


***

1. Соотношение количества вакансий к количеству работодателей 2/1. Конкретных выводов сделать нельзя, так как количество вакансий на одного работодателя может сильно отличаться. Яндекс 100 вакансий в разных городах. ООО "Вася Пупкин" - 0 вакансий. Необходим более глубокий анализ.  
2. Охватываемая область поиска не только Россия, так как в в России только 1117 городов.
3. В данных высокий охват по разным областям деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [189]:
query_4_1 = f'''select 
a.name area,
count(v.id) cnt
from vacancies v 
join areas a on a.id = v.area_id
group by a.name
order by count(v.id) desc
'''

In [190]:
df_4_1 = pd.read_sql_query(query_4_1,connection)
df_4_1

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [191]:
query_4_2 = f'''select count(*)
from vacancies
where salary_from is not null or salary_to is not null
'''

In [192]:
df_4_2 = pd.read_sql_query(query_4_2,connection)
df_4_2

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [193]:
query_4_3 = f'''select 
round(avg(salary_from)),
round(avg(salary_to))
from vacancies
where salary_from is not null or salary_to is not null
'''



In [194]:
df_4_3 = pd.read_sql_query(query_4_3,connection)
df_4_3

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [195]:
query_4_4 = f'''select
schedule,
employment,
count(id)
from vacancies
group by schedule,employment
order by count(id) desc
offset 1 limit 1
'''

In [196]:
df_4_4 = pd.read_sql_query(query_4_4,connection)
df_4_4

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,schedule,employment,count
0,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [197]:
query_4_5 = f'''select 
experience,
count(id)
from vacancies
where experience is not null
group by experience
order by count(id)
'''

In [198]:
df_4_5 = pd.read_sql_query(query_4_5,connection)
df_4_5

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [199]:
bar_4_5 = px.bar(df_4_5,'experience','count',title='График количества вакансий от опыта')
bar_4_5.update_layout(
    xaxis_title= 'Опыт',
    yaxis_title = 'Количество вакансий'
)
bar_4_5.write_html('plots/bar_4_5.html')
display(bar_4_5)

***

1. В первой пятерке по количеству вакансий располагаются столица и города - милионники России и столицы Белорусии и Казахстана.
2. Только у половины соискателей указана желаемая ЗП.
3. В среднем работодатели готовы платить от 70 до 110 тыс.руб.(данные могут быть искажены из-за разных валют)
4. Соискатели чаще ищут удаленую работу с полной занятостью.
5. Работодатели чаще ищут людей с опытом менее 6 лет.


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [200]:
query_5_1 = f'''(select
e.name,
count(v.id) cnt
from employers e
join vacancies v on e.id = v.employer_id
group by e.name
order by count(v.id) desc
limit 1)
union 
(select
e.name,
count(v.id)
from employers e
join vacancies v on e.id = v.employer_id
group by e.name
order by count(v.id) desc
offset 4 limit 1)
order by cnt desc
'''

In [201]:
df_5_1  =  pd. read_sql_query(query_5_1,connection)
df_5_1

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name,cnt
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [202]:
query_5_2 = f'''select
a.name ar,
count(v.id) cnt,
count(e.id) emp
from areas a
left join vacancies v on a.id = v.area_id
join employers e on a.id = e.area
group by a.name 
having count(v.id) = 0
order by emp desc
'''

In [203]:
df_5_2  =  pd. read_sql_query(query_5_2,connection)
df_5_2

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,ar,cnt,emp
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
181,Удобная,0,1
182,Майна (Ульяновская область),0,1
183,Верхнетуломский,0,1
184,Жаркент,0,1


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [204]:
query_5_3 = f'''select
e.name,
count(distinct v.area_id)
from employers e
join vacancies v on e.id = v.employer_id
group by e.name
order by count(distinct v.area_id) desc
'''

In [205]:
df  =  pd. read_sql_query(query_5_3,connection)
df

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [206]:
query_5_4 = f'''select 
count(e.name)
from employers e
left join employers_industries ei on e.id = ei.employer_id
where industry_id is null
'''

In [207]:
df_5_4 =  pd. read_sql_query(query_5_4,connection)
df_5_4

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [208]:
query_5_5 = f'''select 
e.name
from employers e
left join employers_industries ei on e.id = ei.employer_id
where industry_id is not null
group by e.id
having count(ei.industry_id) = 4
order by e.name
offset 2 limit 1 
'''

In [209]:
df_5_5  =  pd. read_sql_query(query_5_5,connection)
df_5_5

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [210]:
query_5_6 = f'''select 
count(distinct e.id)
from employers e
join employers_industries ei on e.id = ei.employer_id
join industries i on ei.industry_id = i.id
group by i.name
having i.name = 'Разработка программного обеспечения'
'''

In [211]:
df_5_6  =  pd. read_sql_query(query_5_6,connection)
df_5_6

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [212]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
responce = requests.get(url)
page = BeautifulSoup(responce.text, 'html.parser')
cities = page.find_all(['td'])
cities_2 = list()
for i in cities:
    if i.find('a') is None:
        continue
    else:
        cities_2.append(i.find('a').text)
million_cities = tuple(cities_2[1:17])
print(million_cities)



('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [213]:
query_5_7 = f'''(select 
a.name,
count(v.name) cnt
from vacancies v
join employers e on e.id = v.employer_id
join areas a on v.area_id = a.id
where e.name = 'Яндекс' and a.name in {million_cities}
group by a.name)
union 
(select 
'total',
count(v.name)
from vacancies v
join employers e on e.id = v.employer_id
join areas a on v.area_id = a.id
where e.name = 'Яндекс' and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')
)
order by cnt
'''

In [214]:
df_5_7  =  pd. read_sql_query(query_5_7,connection)
df_5_7

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

1. Самое большое количество вакансий у Яндекса.
2. По резулльтатам 5.2 регион с наибольшим количеством работодателей и 0 количеством вакансий - Россия. Это конечно же ошибка так как работодатель может указыать областью своего присутствия страну, но для вакансий важен, непосредственно, город.
3. У технологических гигантов (Яндекс, Ростелеком) самый большой охват по регионам.
4. У трети работодателей не указана сфера деятельности, что может привести к путаннице у соискателей, так как многие позиции могут называться одинаково, но специалисты выполняют абсолютно разные функции. 
5. 3553 компании занимают разработкой програмного обеспечения. Это 1/7 работодателй и только 1 из 294 сфер деятельности. 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [215]:
query_6_1 = f'''select count(*)
from vacancies
where LOWER(name) like '%data%' or LOWER(name) like '%данн%'
'''

In [216]:
df_6_1  =  pd.read_sql_query(query_6_1,connection)
df_6_1

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [217]:
comb =['data science','исследователь данных','machine learning','машинн%обучен%']
text_1 = "where ((lower(name) like '%data scientist%' "
text_2 = [f"or lower(name) like '%{i}%'" for i in comb]
text_3 = "or name like '%ML%') and lower(name) not like '%html%')"
ds_search_text = text_1+''.join(text_2)+text_3
junior_search_text = ds_search_text  + " and (lower(name) like '%junior%' or lower(experience) like 'нет опыта' or lower(employment) like 'стажировка')"

In [218]:
query_6_2 = f'''select count(*)
from vacancies
{junior_search_text}
'''


In [219]:
df_6_2  =  pd.read_sql_query(query_6_2,connection)
df_6_2

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [220]:
sql_search_text =ds_search_text+ "and (lower(key_skills) like '%sql%' or lower(key_skills) like '%postgres%')"
query_6_3 = f'''select count(*)
from vacancies
{sql_search_text}
'''

In [221]:
df_6_3 =  pd.read_sql_query(query_6_3,connection)
df_6_3

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [222]:
py_search_text = ds_search_text+ "and (lower(key_skills) like '%python%')"
query_6_4 = f'''select count(*)
from vacancies
{py_search_text}
'''

In [223]:
df_6_4  =  pd.read_sql_query(query_6_4,connection)
df_6_4

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [224]:
key_skills_search_text = ds_search_text +'and key_skills is not null'
query_6_5 = f'''select 
round(avg(length(key_skills)-length(replace(key_skills,CHR(9),''))+1),2)
from vacancies
{key_skills_search_text}
'''

In [225]:
df_6_5  =  pd.read_sql_query(query_6_5,connection)
df_6_5

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [226]:
query_6_6 = f'''select expr, round(avg(avrg))
from (
(select experience expr,
(salary_from+salary_to)/2 avrg
from vacancies
{ds_search_text}
and (salary_from is not null and salary_to is not null)
)
union 
(select experience,
salary_from
from vacancies
{ds_search_text}
and (salary_from is not null and salary_to is null) 
)
union 
(select experience,
salary_to
from vacancies
{ds_search_text}
and (salary_from is null and salary_to is not null) 
)) as er
group by expr
'''


In [227]:
df_6_6  =  pd.read_sql_query(query_6_6,connection)
df_6_6

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,expr,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,141100.0
2,От 3 до 6 лет,228618.0


In [228]:
bar_6_6 = px.bar(df_6_6,'expr','round',title='График распределения ЗП от опыта')
bar_6_6.update_layout(
    xaxis_title= 'Опыт',
    yaxis_title = 'Средняя ЗП'
)
bar_6_6.write_html('plots/bar_6_6.html')
display(bar_6_6)

***

1. 1771 - вакансия имеет отношение к данным.
2. 51 вакансия подходит для junior data scientist
3. В 201 вакансии требуют опыт SQL или Postgresql.
4. Работодатели требую знание python 351 чаще чем SQL.
5. В основном указывают 6 ключевых навыков.
6. Все логично, что с большим стажем работы увеличивается ЗП. Но можно обратить внимание, что требований к опыту более 6 лет в сфере просто нет.

# Общий вывод по проекту

In [229]:
# Хотелось бы увидеть зависимость средней ЗП от направления компании 
query_7_1 = f'''select industry,round(avg(slry))
from(
(select 
i.name industry,
v.salary_from slry
from vacancies v
join employers e on v.employer_id = e.id
join employers_industries ei on ei.employer_id = e.id
join industries i on i.id = ei.industry_id
where salary_from is not null and not (salary_from is not null and salary_to is not null)
)
union all
(select 
i.name,
v.salary_to 
from vacancies v
join employers e on v.employer_id = e.id
join employers_industries ei on ei.employer_id = e.id
join industries i on i.id = ei.industry_id
where salary_to is not null and not (salary_from is not null and salary_to is not null)
)
union all
(select 
i.name,
(v.salary_from + v.salary_to)/2
from vacancies v
join employers e on v.employer_id = e.id
join employers_industries ei on ei.employer_id = e.id
join industries i on i.id = ei.industry_id
where salary_from is not null and salary_to is not null
)
) as r
group by industry
order by avg(slry) desc
'''
df_7_1  =  pd.read_sql_query(query_7_1,connection)
df_7_1

/Users/olegkudrasev/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,industry,round
0,"Агрохимия (продвижение, оптовая торговля)",381429.0
1,Агрохимия (производство),339875.0
2,Игорный бизнес,191905.0
3,Корма для животных (производство),168300.0
4,Зоомагазин,167613.0
...,...,...
286,Фото- и видеоуслуги,40961.0
287,Ритуальные услуги,37401.0
288,"Архив, библиотека, искусствоведение",36858.0
289,Переработка газа,34068.0


In [230]:
tm_7_1 = px.treemap(
    data_frame=df_7_1, #DataFrame
    path=['industry'], #категориальный признак, для которого строится график
    values='round', #параметр, который сравнивается
    height=500, #высота
    width=1000, #ширина
    title='Средняя ЗП в зависимости от индустрии'
)
tm_7_1.write_html('plots/tm_7_1.html')
display(tm_7_1)


По результатам видно, что самые большие ЗП в компаниях, связанных с сельским хозяйством.

1. HH охватывает почти все страны СНГ. Наибольшее количество рабочих мест в столицах и городах - милионниках.
2. В среднем работодатели готовы платить от 70 до 110 тыс.руб.(данные могут быть искажены из-за разных валют)
3. По ЗП лидируют агропромышленность и разработка ПО.
4. Работодатели чаще всего ищут людей с опытом от года до 3 лет, более 6 лет реже всего(скорее всего связанно с тем, что это уже зачастую руководящие должности).
5. В области работы с данными необходимо знать python и sql.